In [21]:
!pip install -q langchain

Amazon Kendra is an intelligent search service powered by machine learning, enabling organizations to provide relevant information to customers and employees, when they need it. Starting today, AWS customers can use Amazon Kendra as a retriever to build retrieval augmented generation (RAG) systems.

Amazon Kendra launches Retrieval API https://js.langchain.com/docs/api/retrievers_amazon_kendra/classes/AmazonKendraRetriever

As Amazon Kendra is a managed services, you can easily setup the search engine and have the index_id on the console, following the tutorial here [Getting started with the Amazon Kendra console](https://docs.aws.amazon.com/kendra/latest/dg/gs-console.html)


In [22]:
from langchain.retrievers import AmazonKendraRetriever
kendra_aws_region = "eu-west-1"

retriever = AmazonKendraRetriever(index_id="c175bf7c-8169-4383-abae-7a2753a95d44",top_k=5,region_name=kendra_aws_region)

In [23]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:  {context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [24]:
from langchain.llms import Bedrock
aws_region = "us-east-1"

inference_modifier = {'max_tokens_to_sample':500, 
                      "temperature":0.0,
                      "top_k":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

bedrock_llm = Bedrock(
    credentials_profile_name= 'cross_account_bedrock_access',
    model_id="anthropic.claude-v2",
    region_name="us-east-1",
    model_kwargs = inference_modifier
)

In [25]:
from langchain.chains import RetrievalQA
import langchain

langchain.verbose = True
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=bedrock_llm, chain_type="stuff", retriever=retriever, chain_type_kwargs=chain_type_kwargs, return_source_documents=True)

In [26]:
query = "Was Xiaomi's financial performance consistent in 2022?"
result = qa(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:  Document Title: 2022_Annual Report_ENG_XIAOMI.pdf
Document Excerpt: 
clarity, and sustainability context, to define our reporting boundary and ensuring proper presentation of our reported information. Materiality: We report and manage material ESG topics which have been identified through stakeholder engagement and materiality assessment exercises. More information can be found in the Stakeholder Engagement and Materiality Assessment sections of this Report. Balance: This Report discloses both positive and negative aspects of our progress to offer an unbiased presentation of Xiaomi’s ESG performance within the reporting period. Quantitative: The methodologies and standards

In [27]:
print(result['result'])

 Based on the excerpts, Xiaomi's financial performance in 2022 faced challenges but remained resilient. The excerpts mention that total revenue was RMB280 billion in 2022, adjusted net profit was RMB8.5 billion including expenses for new initiatives, and each business segment remained resilient amid a challenging macro environment. So while there were challenges, Xiaomi's overall financial performance seems to have remained relatively consistent in 2022 compared to prior years.


In [28]:
print(result['source_documents'])

[Document(page_content='Document Title: 2022_Annual Report_ENG_XIAOMI.pdf\nDocument Excerpt: \nclarity, and sustainability context, to define our reporting boundary and ensuring proper presentation of our reported information. Materiality: We report and manage material ESG topics which have been identified through stakeholder engagement and materiality assessment exercises. More information can be found in the Stakeholder Engagement and Materiality Assessment sections of this Report. Balance: This Report discloses both positive and negative aspects of our progress to offer an unbiased presentation of Xiaomi’s ESG performance within the reporting period. Quantitative: The methodologies and standards used for the calculation of Key Performance Indicators (KPIs), including assumptions, tools, and conversion factors, are described in the corresponding section (if applicable) in this Report. More information can be found in the Stakeholder Engagement and Materiality Assessment sections of t